## Проект по SQL

### Цель проекта

Наша задача — проанализировать базу данных.

### Этапы проекта

- [Подключиться к базе данных](#connection)
- [Исследовать таблицы — вывести первые строки](#researching)
- [Анализ базы данных](#analysis)
    - Посчитать, сколько книг вышло после 1 января 2000 года
    - Для каждой книги посчитать количество обзоров и среднюю оценку
    - Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры
    - Определить автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками
    - Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
- [Вывод](#conclusion)

### Схема данных
<a id="scheme"></a>

<img src="https://pictures.s3.yandex.net/resources/scheme_1589269096.png">

### Подключение к базе данных
<a id="connection"></a>

Подключимся к базе данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
db_config = {'user': '',
'pwd': '',
'host': '',
'port': , 
'db': ''}   #удалил в целях безопасности

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [5]:
query = "SELECT * FROM books LIMIT 10"

In [6]:
pd.io.sql.read_sql(query, con = engine, index_col='book_id') 

,author_id,title,num_pages,publication_date,publisher_id
book_id,,,,,
1,546,'Salem's Lot,594,2005-11-01,93
2,465,1 000 Places to See Before You Die,992,2003-05-22,336
3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
5,125,1776,386,2006-07-04,268
6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
9,563,A Beautiful Mind,461,2002-02-04,104


Мы успешно подключились к базе данных. Выведем все таблицы для проверки

### Исследуем все таблицы
<a id="researching">

Для удобства напишем функцию, которая выводит наш запрос

In [7]:
def return_sql(query):
    return pd.io.sql.read_sql(query, con = engine, parse_dates=['publication_date'])

Запишем названия наших таблиц в список, а затем пройдемся по нему с вызовом функции

In [8]:
tables = ['books', 'authors', 'ratings', 'reviews', 'publishers']
for table in tables:
    display(return_sql(f'SELECT * FROM {table} LIMIT 10'))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


Мы просмотрели все таблицы, они соответствуют схеме данных. Можем переходить к анализу БД

### Анализ БД
<a id="analysis">

**1) Посчитаем, сколько книг вышло после 1 января 2000 года**<br>
[Посмотреть схему БД](#scheme)

In [9]:
query = '''
        SELECT COUNT(*) as count_books
        FROM books 
        WHERE publication_date > '2000-01-01'
        '''
return_sql(query)

,count_books
0,819


После 1 января 2000 года было опубликовано 819 книг

**2) Для каждой книги посчитаем количество обзоров и среднюю оценку** <br>
[Посмотреть схему БД](#scheme)

In [10]:
query = '''
        SELECT DISTINCT b.book_id,
        b.title,
        ROUND(AVG(r.rating),2) as avg_rating,
        COUNT(DISTINCT re.text) as count_reviews
        FROM books b
        LEFT JOIN ratings r ON  r.book_id = b.book_id
        LEFT JOIN reviews re ON re.book_id = b.book_id
        GROUP BY 1
        ORDER BY 3 DESC, 4 DESC
        '''
return_sql(query)

,book_id,title,avg_rating,count_reviews
0,17,A Dirty Job (Grim Reaper #1),5.00,4
1,444,Moneyball: The Art of Winning an Unfair Game,5.00,3
2,553,School's Out—Forever (Maximum Ride #2),5.00,3
3,20,A Fistful of Charms (The Hollows #4),5.00,2
4,55,A Woman of Substance (Emma Harte Saga #1),5.00,2
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3
996,202,Drowning Ruth,2.00,3
997,316,His Excellency: George Washington,2.00,2
998,371,Junky,2.00,2


По всем книгам у нас рейтинг от 5 и вплоть до 1,5.

**3) Определим издательство, которое выпустило наибольшее число книг толще 50 страниц**<br>
[Посмотреть схему БД](#scheme)

In [11]:
query = '''
        SELECT p.publisher, 
        COUNT(b.book_id) as count_books
        FROM publishers p
        JOIN books b ON p.publisher_id = b.publisher_id
        WHERE b.num_pages>50
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 1
        '''
return_sql(query)

,publisher,count_books
0,Penguin Books,42


Издательство Penguin Books выпустило 42 книги!

**4) Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками**<br>
[Посмотреть схему БД](#scheme)

In [12]:
query = '''
        WITH good_books as
        (
        SELECT b.book_id
        FROM books b
        JOIN ratings r ON r.book_id = b.book_id
        GROUP BY 1
        HAVING COUNT(r.book_id)>=50
        ORDER BY COUNT(r.book_id) DESC
        )
        
        SELECT a.author,
        AVG(tab.avg_rating) as avg_rating
        FROM books b
        JOIN
            (SELECT r1.book_id,
            ROUND(AVG(r1.rating),2) as avg_rating
            FROM ratings r1
            JOIN good_books gb ON gb.book_id = r1.book_id
            GROUP BY 1) tab ON tab.book_id = b.book_id
        JOIN authors a ON a.author_id = b.author_id
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 1
        '''
return_sql(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.285


Джоан Роулинг возглавляет наш список!

**5) Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**<br>
[Посмотреть схему БД](#scheme)

In [13]:
query = '''
        WITH our_table as (
        SELECT re.username,
        COUNT(*) as count_reviews
        FROM reviews re
        WHERE re.username IN(
            SELECT r.username
            FROM ratings r
            GROUP BY 1
            HAVING COUNT(*) > 50
            ORDER BY COUNT(*) DESC
        )
        GROUP BY 1
        ) 
        
        SELECT ROUND(AVG(our_table.count_reviews), 0) as avg_count_reviews
        FROM our_table

        '''
return_sql(query)

,avg_count_reviews
0,24.0


Пользователи, поставившие более 50 оценок, в среднем написали около 24 обзоров на человека

### Вывод
<a id='conclusion'></a>

База данных изучена! Было написано несколько запросов, которые помогли нам получить представление о БД. Оказалось, что после 1 января 2000 года вышло 819 книг, а издательство Penguin Books выпустило больше всех книг.